In [24]:
import os
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModel, pipeline
from transformers.pipelines.pt_utils import KeyDataset
from datasets import Dataset
from tqdm.notebook import tqdm

In [48]:
import multiprocessing

try:
    multiprocessing.set_start_method('fork', force=True)
except RuntimeError:
    pass # Already set

# Load in data

In [4]:
# get classification data
CLASSES_PATH = os.path.expanduser("~/Projects/deep-learning-26wi/datasets/combined_hglt_soc.tsv")
INTERACTIONS_PATH = os.path.expanduser("~/Projects/deep-learning-26wi/datasets/agnostic_interaction_descriptions.tsv")


In [5]:
classes = pd.read_csv(CLASSES_PATH, delimiter="\t")
classes.head()

,ID,TYPE OF DATA FOR HGLT,HIERARCHY,Name FOR HGLT,TYPE OF DATA FOR SOC,BROADER ID,Name FOR SOC
0,10000073,type,HGLT,Abdominal hernias and other abdominal wall con...,broader,10017947,Gastrointestinal disorders
1,10000211,type,HGLT,Abortions and stillbirth,broader,10036585,"Pregnancy, puerperium and perinatal conditions"
2,10000485,type,HGLT,Acid-base disorders,broader,10027433,Metabolism and nutrition disorders
3,10000546,type,HGLT,Metabolism disorders NEC,broader,10027433,Metabolism and nutrition disorders
4,10001302,type,HGLT,Adjustment disorders (incl subtypes),broader,10037175,Psychiatric disorders


In [6]:
interactions = pd.read_csv(INTERACTIONS_PATH, delimiter="\t")
interactions_dataset = Dataset.from_pandas(interactions[["description"]])
interactions.head()

,id,description,count
0,0,may increase the anticoagulant activities of,11149
1,1,The risk or severity of bleeding and hemorrhag...,24244
2,2,The risk or severity of gastrointestinal bleed...,10602
3,3,The risk or severity of bleeding and bruising ...,1864
4,4,The risk or severity of bleeding can be increa...,51797


In [7]:
id2label = {}
label2id = {}
for _, row in classes.iterrows():
    class_id = row["ID"]
    class_name = row["Name FOR HGLT"]
    id2label[class_id] = class_name
    label2id[class_name] = class_id

In [8]:
def get_blackwell_device():
    if not torch.cuda.is_available():
        return torch.device("cpu")
    
    for i in range(torch.cuda.device_count()):
        # capability returns (major, minor) e.g., (12, 0) for sm_120
        major, minor = torch.cuda.get_device_capability(i)
        if major == 12:
            print(f"Found Blackwell GPU at index {i}: {torch.cuda.get_device_name(i)}")
            return torch.device(f"cuda:{i}")
            
    return torch.device("cuda:0") # Fallback to first GPU

DEVICE = get_blackwell_device()
DEVICE

Found Blackwell GPU at index 0: NVIDIA GB10


device(type='cuda', index=0)

In [55]:
model = AutoModel.from_pretrained("gsarti/biobert-nli")
model = model.to(DEVICE)
tokenizer = AutoTokenizer.from_pretrained("gsarti/biobert-nli", device=DEVICE)
# model.config.id2label = id2label
# model.config.label2id = label2id
classify = pipeline(
    task="zero-shot-classification",
    model="gsarti/biobert-nli",
    device=DEVICE,
    # candidate_labels=list(label2id.keys())
)

Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

BertForSequenceClassification LOAD REPORT from: gsarti/biobert-nli
Key               | Status  | 
------------------+---------+-
classifier.weight | MISSING | 
classifier.bias   | MISSING | 

Notes:
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.
Failed to determine 'entailment' label id from the label2id mapping in the model config. Setting to -1. Define a descriptive label2id mapping in the model config to ensure correct outputs.


In [64]:
results = []

# Using a generator for memory efficiency
for out in tqdm(classify(
    KeyDataset(interactions_dataset, "description"),
    batch_size=128,
    candidate_labels=list(label2id.keys())
)):
    results.append(out)

  0%|          | 0/6 [00:00<?, ?it/s]

/home/anaya/Projects/deep-learning-26wi/.venv/lib/python3.14/site-packages/transformers/pipelines/pt_utils.py:271: UserWarning: Length of IterableDataset <transformers.pipelines.pt_utils.PipelineChunkIterator object at 0xe4efa72d7c50> was reported to be 683(when accessing len(dataloader)), but 684 samples have been fetched. 
  processed = self.infer(next(self.iterator), **self.params)
/home/anaya/Projects/deep-learning-26wi/.venv/lib/python3.14/site-packages/transformers/pipelines/pt_utils.py:271: UserWarning: Length of IterableDataset <transformers.pipelines.pt_utils.PipelineChunkIterator object at 0xe4efa72d7c50> was reported to be 683(when accessing len(dataloader)), but 685 samples have been fetched. 
  processed = self.infer(next(self.iterator), **self.params)
/home/anaya/Projects/deep-learning-26wi/.venv/lib/python3.14/site-packages/transformers/pipelines/pt_utils.py:271: UserWarning: Length of IterableDataset <transformers.pipelines.pt_utils.PipelineChunkIterator object at 0xe4e

In [65]:
interactions["results"] = results
def process_res(res):
    label, score = sorted(list(zip(res["labels"], res["scores"])), key=lambda p: p[1], reverse=True)[0]
    return pd.Series([label, score])

interactions[["Top Label", "Top Score"]] = interactions["results"].apply(process_res)
interactions["results"] = interactions.pop("results")
interactions.to_csv("~/Projects/deep-learning-26wi/datasets/classified_interactions.tsv", sep="\t", index=False)

In [68]:
combined = pd.read_csv("~/Projects/deep-learning-26wi/datasets/combined_hglt_soc.tsv", delimiter="\t")
combined.head()

,ID,TYPE OF DATA FOR HGLT,HIERARCHY,Name FOR HGLT,TYPE OF DATA FOR SOC,BROADER ID,Name FOR SOC
0,10000073,type,HGLT,Abdominal hernias and other abdominal wall con...,broader,10017947,Gastrointestinal disorders
1,10000211,type,HGLT,Abortions and stillbirth,broader,10036585,"Pregnancy, puerperium and perinatal conditions"
2,10000485,type,HGLT,Acid-base disorders,broader,10027433,Metabolism and nutrition disorders
3,10000546,type,HGLT,Metabolism disorders NEC,broader,10027433,Metabolism and nutrition disorders
4,10001302,type,HGLT,Adjustment disorders (incl subtypes),broader,10037175,Psychiatric disorders


In [71]:
full = interactions.merge(combined, left_on="Top Label", right_on="Name FOR HGLT")
full.to_csv("~/Projects/deep-learning-26wi/datasets/classified_interactions_full.tsv", index=False)
full

,id,description,count,Top Label,Top Score,results,ID,TYPE OF DATA FOR HGLT,HIERARCHY,Name FOR HGLT,TYPE OF DATA FOR SOC,BROADER ID,Name FOR SOC
0,0,may increase the anticoagulant activities of,11149,Vascular haemorrhagic disorders,0.006281,{'sequence': 'may increase the anticoagulant a...,10047075,type,HGLT,Vascular haemorrhagic disorders,broader,10047065,Vascular disorders
1,1,The risk or severity of bleeding and hemorrhag...,24244,Vascular haemorrhagic disorders,0.003830,{'sequence': 'The risk or severity of bleeding...,10047075,type,HGLT,Vascular haemorrhagic disorders,broader,10047065,Vascular disorders
2,2,The risk or severity of gastrointestinal bleed...,10602,Gastrointestinal haemorrhages NEC,0.004053,{'sequence': 'The risk or severity of gastroin...,10017959,type,HGLT,Gastrointestinal haemorrhages NEC,broader,10017947,Gastrointestinal disorders
3,3,The risk or severity of bleeding and bruising ...,1864,"Product quality, supply, distribution, manufac...",0.003815,{'sequence': 'The risk or severity of bleeding...,10077537,type,HGLT,"Product quality, supply, distribution, manufac...",broader,10077536,Product issues
4,4,The risk or severity of bleeding can be increa...,51797,Vascular haemorrhagic disorders,0.003836,{'sequence': 'The risk or severity of bleeding...,10047075,type,HGLT,Vascular haemorrhagic disorders,broader,10047065,Vascular disorders
...,...,...,...,...,...,...,...,...,...,...,...,...,...
699,678,can cause a decrease in the absorption of sul...,1,Malabsorption conditions,0.003957,{'sequence': 'can cause a decrease in the abso...,10025477,type,HGLT,Malabsorption conditions,broader,10017947,Gastrointestinal disorders
700,679,The risk or severity of CNS depression can be ...,1,Acid-base disorders,0.004721,{'sequence': 'The risk or severity of CNS depr...,10000485,type,HGLT,Acid-base disorders,broader,10027433,Metabolism and nutrition disorders
701,680,etexilate may increase the anticoagulant activ...,1,Vascular haemorrhagic disorders,0.006029,{'sequence': 'etexilate may increase the antic...,10047075,type,HGLT,Vascular haemorrhagic disorders,broader,10047065,Vascular disorders
702,681,The risk or severity of immunosuppression can ...,2,Immunodeficiency syndromes,0.004551,{'sequence': 'The risk or severity of immunosu...,10021460,type,HGLT,Immunodeficiency syndromes,broader,10021428,Immune system disorders


In [75]:
interactions = pd.read_csv("~/Projects/deep-learning-26wi/datasets/interactions.tsv", delimiter="\t")
og_linked_full = interactions.merge(full, left_on="agnostic_interaction_description", right_on="description").drop("description", axis=1)
og_linked_full.head()

,drug_primary_id,drug_name,other_drug_id,other_drug_name,interaction_description,agnostic_interaction_id,agnostic_interaction_description,id,count,Top Label,Top Score,results,ID,TYPE OF DATA FOR HGLT,HIERARCHY,Name FOR HGLT,TYPE OF DATA FOR SOC,BROADER ID,Name FOR SOC
0,DB00001,Lepirudin,DB06605,Apixaban,Apixaban may increase the anticoagulant activi...,0,may increase the anticoagulant activities of,0,11149,Vascular haemorrhagic disorders,0.006281,{'sequence': 'may increase the anticoagulant a...,10047075,type,HGLT,Vascular haemorrhagic disorders,broader,10047065,Vascular disorders
1,DB00001,Lepirudin,DB06695,Dabigatran etexilate,Dabigatran etexilate may increase the anticoag...,0,may increase the anticoagulant activities of,0,11149,Vascular haemorrhagic disorders,0.006281,{'sequence': 'may increase the anticoagulant a...,10047075,type,HGLT,Vascular haemorrhagic disorders,broader,10047065,Vascular disorders
2,DB00001,Lepirudin,DB01254,Dasatinib,The risk or severity of bleeding and hemorrhag...,1,The risk or severity of bleeding and hemorrhag...,1,24244,Vascular haemorrhagic disorders,0.003830,{'sequence': 'The risk or severity of bleeding...,10047075,type,HGLT,Vascular haemorrhagic disorders,broader,10047065,Vascular disorders
3,DB00001,Lepirudin,DB01609,Deferasirox,The risk or severity of gastrointestinal bleed...,2,The risk or severity of gastrointestinal bleed...,2,10602,Gastrointestinal haemorrhages NEC,0.004053,{'sequence': 'The risk or severity of gastroin...,10017959,type,HGLT,Gastrointestinal haemorrhages NEC,broader,10017947,Gastrointestinal disorders
4,DB00001,Lepirudin,DB01586,Ursodeoxycholic acid,The risk or severity of bleeding and bruising ...,3,The risk or severity of bleeding and bruising ...,3,1864,"Product quality, supply, distribution, manufac...",0.003815,{'sequence': 'The risk or severity of bleeding...,10077537,type,HGLT,"Product quality, supply, distribution, manufac...",broader,10077536,Product issues


In [ ]:
og_linked_full.to_csv("~/Projects/deep-learning-26wi/datasets/original_interactions_linked_to_full_classes.tsv", index=False)

In [56]:
classify(
    "The risk or severity of bleeding and hemorrhage can be increased when  is combined with",
    candidate_labels=list(label2id.keys())
)

{'sequence': 'The risk or severity of bleeding and hemorrhage can be increased when  is combined with',
 'labels': ['Vascular haemorrhagic disorders',
  'Central nervous system vascular disorders',
  'Increased intracranial pressure and hydrocephalus',
  'Coagulopathies and bleeding diatheses (excl thrombocytopenic)',
  'Platelet disorders',
  'Vascular injuries',
  'Exposures, chemical injuries and poisoning',
  'Product quality, supply, distribution, manufacturing and quality system issues',
  'Pulmonary vascular disorders',
  'Venous varices',
  'Injuries by physical agents',
  'Legal issues',
  'Embolism and thrombosis',
  'Device issues',
  'Placental, amniotic and cavity disorders (excl haemorrhages)',
  'Haemolyses and related conditions',
  'Nervous system, skull and spine therapeutic procedures',
  'Eye therapeutic procedures',
  'Haematological and lymphoid tissue therapeutic procedures',
  'Procedural related injuries and complications NEC',
  'Medication errors and other pr

In [43]:
interactions["results"].iloc[0]

{'sequence': 'may increase the anticoagulant activities of',
 'labels': ['Soft tissue neoplasms benign',
  'Reproductive neoplasms male benign',
  'Reproductive neoplasms female benign',
  'Nervous system neoplasms benign',
  'Cutaneous neoplasms benign',
  'Skeletal neoplasms benign',
  'Gastrointestinal neoplasms benign',
  'Endocrine neoplasms benign',
  'Soft tissue neoplasms malignant and unspecified',
  'Hepatic and biliary neoplasms benign',
  'Ocular sensory symptoms NEC',
  'Respiratory and mediastinal neoplasms benign (excl mesotheliomas)',
  'Reproductive neoplasms female malignant and unspecified',
  'Breast neoplasms benign (incl nipple)',
  'Skeletal neoplasms malignant and unspecified',
  'Head and neck therapeutic procedures',
  'Skin neoplasms malignant and unspecified',
  'Miscellaneous and site unspecified neoplasms benign',
  'Oral soft tissue conditions',
  'Reproductive neoplasms male malignant and unspecified',
  'Cranial nerve disorders (excl neoplasms)',
  'Sof

In [ ]:
res = pipe(
    "The risk or severity of Thrombosis can be increased when  is combined with KRN-7000",
    candidate_labels=[p[1] for p in hlgt_name_pairs]
)
sorted(zip(res["labels"], res["scores"]), key=lambda p: p[-1], reverse=True)

[('Embolism and thrombosis', 0.004876430612057447),
 ('Coagulopathies and bleeding diatheses (excl thrombocytopenic)',
  0.004614297300577164),
 ('Vascular haemorrhagic disorders', 0.004389713518321514),
 ('Heart failures', 0.004375307355076075),
 ('Fatal outcomes', 0.004364895634353161),
 ('Device issues', 0.004344609100371599),
 ('Product quality, supply, distribution, manufacturing and quality system issues',
  0.004309582058340311),
 ('Platelet disorders', 0.004270956385880709),
 ('Red blood cell disorders', 0.004232048988342285),
 ('Haemolyses and related conditions', 0.0042101615108549595),
 ('Venous varices', 0.004183194600045681),
 ('Administration site reactions', 0.004182442557066679),
 ('Tongue conditions', 0.004112924449145794),
 ('Medication errors and other product use errors and issues',
  0.004079894628375769),
 ('Haematology investigations (incl blood groups)', 0.0040760342963039875),
 ('Aneurysms and artery dissections', 0.004060270264744759),
 ('Vascular injuries', 0